# <b>Fórum avaliativo de Processamento Paralelo e Distribuído</b>
**Objetivo:**<br>
Demonstrar o uso de paralelismo para a multiplicação de matrizes<br>

**Grupo**: Átila Madureira, Gabriel Lemos, Letycia Milene, Vinícius Guimarães

In [1]:
#inicialmente instalar a dependência necessária para essa tarefa
!pip install mpi4py
!pip install numba

     |████████████████████████████████| 2.5 MB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185301 sha256=60f92351e573f9030629bfe334e049a8744aefbc150ebca656f510b034db55ba
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


Aqui geramos 10 matrizes de 512x512 valores aleatórios entre **0** e **1024**, e posteriormente a função de multiplicação é definida. A multiplicação de matrizes na biblioteca Numpy é extremamente rápida.

In [11]:


#geração de matrizes aleatórias
matrix_n = 512

#valores máximo para geração de números aleatórios
rand_min = 0
rand_max = 1024 

arr1 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr2 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr3 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr4 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr5 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr6 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr7 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr8 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr9 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))
arr10 = np.random.randint(rand_min,rand_max, size=(matrix_n,matrix_n))

#listas dos objetos a serem multiplicados
matrix_set_1 = [arr1,arr2,arr3,arr4,arr5]
matrix_set_2 = [arr6,arr7,arr8,arr9,arr10]

import time

def multiply(a, b):
  return a@b

In [42]:
import numba
import numpy as np

#valores máximo para geração de números aleatórios
rand_min = 0
rand_max = 255
list_length = 20000

list1 = np.random.randint(rand_min,rand_max, size=(list_length))
list2 = np.random.randint(rand_min,rand_max, size=(list_length))

@numba.njit
def media(valores):
  soma = 0
  for i in numba.prange(len(valores)):
    soma += valores[i]
  return soma/ float(len(valores))

@numba.njit
def covariancia(x, media_x, y ,media_y):
  covar = 0.0
  for i in numba.prange(len(x)):
    covar += (x[i] - media_x) * (y[i] - media_y) 

  return covar

@numba.njit
def variancia(valores, media):
  soma = 0
  for i in numba.prange(len(valores)):
    soma += (valores[i] - media) ** 2

  return soma

@numba.njit(parallel=True)
def coef_regressao_linear(x ,y):
  x_media = media(x)
  y_media = media(y)
  b1 = covariancia(x, x_media, y, y_media) / variancia(x, x_media)
  b0 = y_media - b1 * x_media
  return [b0, b1]

In [44]:
start = time.perf_counter()

coef_regressao_linear(list1,list2)

finish = time.perf_counter()
print(f'Calculo com paralelismo finalizado em {finish-start} segundo(s)')


Calculo com paralelismo finalizado em 0.0002148039984604111 segundo(s)


Por fim, o cálculo de matrizes é feito paralelamente e depois em sequência, e os tempos são medidos.

In [ ]:
from mpi4py import MPI
from mpi4py.futures import MPICommExecutor

start = time.perf_counter()
with MPICommExecutor(MPI.COMM_WORLD, root=0) as executor:
    if executor is not None:
      future = executor.map(multiply, matrix_set_1, matrix_set_2)
    #print(future.result())   

finish = time.perf_counter()
print(f'Calculo por paralelismo finalizado em {finish-start} segundo(s)')

start = time.perf_counter()
for i in range(5):
  results = multiply(matrix_set_1[i], matrix_set_2[i])

finish = time.perf_counter()

print(f'Calculo por sequencia finalizado em {finish-start} segundo(s)')


Calculo por paralelismo finalizado em 2.7301639350000073 segundo(s)
Calculo por sequencia finalizado em 2.6697849539999936 segundo(s)
